In [1]:
# Import libraries
library(shiny)
library(shinythemes)

In [2]:
# Define UI 
ui <- fluidPage(theme = shinytheme("united"),
                navbarPage("BMI Calculator:",
                           
                           tabPanel("Home",
                                    # Input values
                                    sidebarPanel(
                                      HTML("<h3>Input parameters</h3>"),
                                      # Slider and Text Input for Height
                                      sliderInput("height", 
                                                  label = "Height (cm)", 
                                                  value = 175, 
                                                  min = 40, 
                                                  max = 250),
                                      textInput("height_text", 
                                                label = "Enter Height (cm)", 
                                                value = 175),
                                      
                                      # Slider and Text Input for Weight
                                      sliderInput("weight", 
                                                  label = "Weight (kg)", 
                                                  value = 70, 
                                                  min = 20, 
                                                  max = 100),
                                      textInput("weight_text", 
                                                label = "Enter Weight (kg)", 
                                                value = 70),
                                      
                                      actionButton("submitbutton", 
                                                   "Submit", 
                                                   class = "btn btn-primary")
                                    ),
                                    
                                    mainPanel(
                                      tags$label(h3('Status/Output')), # Status/Output Text Box
                                      verbatimTextOutput('contents'),
                                      
                                      # BMI Result Table
                                      tableOutput('tabledata'),
                                      
                                      # BMI Classification and Scale
                                      tags$h4("BMI Classification"),
                                      uiOutput("bmi_category"),
                                      uiOutput("bmi_scale") 
                                    ) # mainPanel()
                                    
                           ), #tabPanel(), Home
                           
                           tabPanel("About", 
                                    titlePanel("About"), 
                                    div(includeMarkdown("about.md"), 
                                        align="justify")
                           ) #tabPanel(), About
                           
                ) # navbarPage()
) # fluidPage()

In [3]:
server <- function(input, output, session) {
  
  # Update slider value when text input changes
  observeEvent(input$height_text, {
    updateSliderInput(session, "height", value = as.numeric(input$height_text))
  })
  
  observeEvent(input$weight_text, {
    updateSliderInput(session, "weight", value = as.numeric(input$weight_text))
  })
  
  # Input Data and BMI Calculation (only triggered by the submit button)
  datasetInput <- eventReactive(input$submitbutton, {  
    
    bmi <- input$weight / ( (input$height / 100) * (input$height / 100) )
    bmi <- round(bmi, 2)
    bmi_data <- data.frame(BMI = bmi)
    
    # BMI Classification
    if (bmi < 18.5) {
      category <- "Underweight"
      color <- "blue"
    } else if (bmi >= 18.5 && bmi < 25) {
      category <- "Healthy Weight"
      color <- "green"
    } else if (bmi >= 25 && bmi < 30) {
      category <- "Overweight"
      color <- "orange"
    } else {
      category <- "Obese"
      color <- "red"
    }
    
    # Return BMI and category information
    return(list(BMI = bmi_data, category = category, color = color))
  })
  
  # Status/Output Text Box
  output$contents <- renderPrint({
    if (input$submitbutton > 0) { 
      isolate("Calculation complete.") 
    } else {
      return("Server is ready for calculation.")
    }
  })
  
  # Prediction results table
  output$tabledata <- renderTable({
    if (input$submitbutton > 0) { 
      isolate(datasetInput()$BMI) 
    } 
  })
  
  # BMI Category
  output$bmi_category <- renderUI({
    if (input$submitbutton > 0) {
      category <- datasetInput()$category
      tagList(tags$p(paste("Your BMI falls under:", category)))
    }
  })
  
  # BMI Scale with Dot/Arrow and Numeric Labels
  output$bmi_scale <- renderUI({
    if (input$submitbutton > 0) {
      bmi_value <- datasetInput()$BMI$BMI
      
      # Fixed sections on the scale
      scale_width <- 100  # Full width of the scale (100%)
      sections <- c(18.5, 25, 30)  # The limits for each BMI category
      
      # Calculate the position of the dot/arrow on the scale
      position <- (bmi_value / 40) * scale_width  # Scale is from 0 to 40 for BMI values
      
      # Create the scale
      tagList(
        # Scale div
        div(
          style = paste0("width: 100%; height: 30px; background-color: lightgrey; position: relative;"),
          
          # Underweight section
          div(
            style = paste0("width: ", (sections[1] / 40) * scale_width, "%; height: 100%; background-color: blue; float: left;"),
            tags$p("Underweight", style = "position: absolute; left: 0%; top: 35px; color: white; font-size: 12px;")
          ),
          
          # Healthy Weight section
          div(
            style = paste0("width: ", ((sections[2] - sections[1]) / 40) * scale_width, "%; height: 100%; background-color: green; float: left;"),
            tags$p("Healthy Weight", style = "position: absolute; left: 25%; top: 35px; color: white; font-size: 12px;")
          ),
          
          # Overweight section
          div(
            style = paste0("width: ", ((sections[3] - sections[2]) / 40) * scale_width, "%; height: 100%; background-color: orange; float: left;"),
            tags$p("Overweight", style = "position: absolute; left: 50%; top: 35px; color: white; font-size: 12px;")
          ),
          
          # Obese section
          div(
            style = paste0("width: ", ((40 - sections[3]) / 40) * scale_width, "%; height: 100%; background-color: red; float: left;"),
            tags$p("Obese", style = "position: absolute; left: 75%; top: 35px; color: white; font-size: 12px;")
          ),
          
          # Dot or Arrow indicating the BMI position
          div(
            style = paste0("position: absolute; left: ", position, "%; top: 10px; height: 15px; width: 15px; background-color: black; border-radius: 50%;"),
          ),
          
          # Adding numeric labels to act like a ruler
          tags$p(style = "position: absolute; left: 0%; top: 40px; font-size: 12px;", "0"),
          tags$p(style = paste0("position: absolute; left: ", (18.5 / 40) * scale_width, "%; top: 40px; font-size: 12px;"), "18.5"),
          tags$p(style = paste0("position: absolute; left: ", (25 / 40) * scale_width, "%; top: 40px; font-size: 12px;"), "25"),
          tags$p(style = paste0("position: absolute; left: ", (30 / 40) * scale_width, "%; top: 40px; font-size: 12px;"), "30"),
          tags$p(style = paste0("position: absolute; left: 100%; top: 40px; font-size: 12px;"), "40")
        ),
        
        # BMI value
        tags$p(paste("BMI: ", round(bmi_value, 2)))
      )
    }
  })
}

In [ ]:
# Create Shiny App
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:3441

